In [ ]:
%reload_ext autoreload
%autoreload 2

import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pickle

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres

In [ ]:
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH6.4b_S20220419'
flexilims_session = flz.get_flexilims_session(project)
exp_session = flz.get_entity(
    datatype="session", name=session_name, flexilims_session=flz_session
)
recording_type='two_photon'
recordings = flz.get_entities(
    datatype="recording",
    origin_id=exp_session["id"],
    query_key="recording_type",
    query_value=recording_type,
    flexilims_session=flz_session,
)
recording = flz.get_entity(
    datatype="recording",
    name=recordings.iloc[0].name,
    flexilims_session=flz_session
)
# vs_df_all, trials_df_all = spheres.sync_all_recordings(
#     session_name=session_name,
#     flexilims_session=flz_session,
#     project=project,
#     filter_datasets={'anatomical_only':3},
#     recording_type="two_photon",
#     protocol_base="SpheresPermTubeReward",
#     photodiode_protocol=2,
#     return_volumes=True,
# )
vs_df = synchronisation.generate_vs_df(
    recording=recording,
    photodiode_protocol=2,
    flexilims_session=None,
    project='hey2_3d-vision_foodres_20220101',
)

imaging_df = synchronisation.generate_imaging_df(
    vs_df=vs_df,
    recording=recording,
    flexilims_session=flz_session,
    filter_datasets={'anatomical_only':3},
    return_volumes=True
)

imaging_df = spheres.format_imaging_df(
    recording=recording,
    imaging_df=imaging_df
)

trials_df = spheres.generate_trials_df(
    recording=recording,
    imaging_df=imaging_df
)

trials_df = spheres.search_param_log_trials(
    recording=recording,
    trials_df=trials_df, 
    flexilims_session=flexilims_session)

harp_ds = flz.get_datasets(
    flexilims_session=flexilims_session,
    origin_name=recording.name,
    dataset_type="harp",
    allow_multiple=False,
    return_dataseries=False,
)
paramlog_path = harp_ds.path_full / harp_ds.csv_files["NewParams"]
param_log = pd.read_csv(paramlog_path)

frames = spheres.regenerate_frames(
    frame_times=imaging_df.imaging_harptime,
    trials_df=trials_df,
    vs_df=vs_df,
    param_logger=param_log,
    time_column="HarpTime",
    resolution=1,
    sphere_size=10,
    azimuth_limits=(-120, 120),
    elevation_limits=(-40, 40),
    verbose=True,
    output_datatype="int16",
    output=None,
)


In [ ]:
plt.imshow(frames[10000], origin='lower')